# Analyse baseline model 

In this document we will create a baseline model that is the simplest solution for us.
This baseline model was created with a CNN network. The structure of the model can be found in [this](https://github.com/BB8-2020/EmpathicRobot/blob/baseline-model/baseline-model/baseline_model.py) file. 

In [1]:
# import the model python code 
import baseline_model as ml

Lets us start with creating the model:

In [2]:
baseline_model = ml.create_model()

The model summary looks as here below:

### Build model 
__1. Conv Layer__

The first layer consists of 32 3x3 filters with ReLU. We set the input of this layer equal to the shape of the train data, which is (48, 48, 1).

We leave the stride and padding at the default value. In the next model we are going to play with this but for now we keep it at 1 and "vaild".

We do add a Batch normalization. The output of this layer (the activation map) is (46, 46, 32).

__2. Conv Layer__

The second layer consists of 64 filters of 3x3 and here we apply batch normalization. The output of this layer woudld be (44, 44, 64) 

__3. Conv Layer__

The last conv layer consists of 128 filters of 3x3 and we also apply a max pooling of (2,2) that produce output shape of (21, 21, 128). We also add a batch normalization to this layer. 

To this layer we add a flatten option, that means the output shape of this layer would be (56448) 

__4. fully connected layer__

This layer takes (56448) as input. The output shape of this layer is the probability of 2 classes. 

In [3]:
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 46, 46, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 32)        128       
_________________________________________________________________
conv2 (Conv2D)               (None, 44, 44, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
conv3 (Conv2D)               (None, 42, 42, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 42, 42, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 21, 21, 128)       0

Looks good, time to compile!

## Model complie and train

To compile the model we use Adam optimaizer and categorical_crossentropy as los function. Let us now train the model.

In [4]:
# history = ml.train_model(model= baseline_model, batch_size = 10, epochs=5, vs = 0.2)
# ik kan dit niet runnen want he mn laptop. 

## Results

In [5]:
# hier beschrijf je het resultaat van het model, teken hierbij een mooi plot van de acc en loss fucntie 

In [6]:
#Deze functie mag je ergens anders platsen + docuemteren 
def plot_results(history):
    """Plot the results of the fit function of the model.
       takes history: A nummpy array as input."""
    
    plt.plot(history.history['accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()

In [7]:
#plot_results(history)

## Conclusion 

In [8]:
# hier zeg we wat de resultaten van het model betekennen en waarvoor we deze resultaten precies gaan gebruiken. 